In [1]:
import pandas as pd

df = pd.read_csv("../public/data/smart_mobility_dataset.csv", parse_dates=["Timestamp"])

In [2]:
df.head()

,Timestamp,Latitude,Longitude,Vehicle_Count,Traffic_Speed_kmh,Road_Occupancy_%,Traffic_Light_State,Weather_Condition,Accident_Report,Sentiment_Score,Ride_Sharing_Demand,Parking_Availability,Emission_Levels_g_km,Energy_Consumption_L_h,Traffic_Condition
0,2024-03-01 00:00:00,40.842275,-73.703149,205,49.893435,82.652780,Yellow,Clear,0,-0.609199,2,45,450.760055,19.574337,High
1,2024-03-01 00:05:00,40.831119,-73.987354,202,22.383965,45.829298,Green,Clear,0,0.965442,16,1,321.800341,5.385554,High
2,2024-03-01 00:10:00,40.819549,-73.732462,252,46.889699,82.772465,Green,Rain,0,0.289660,16,49,231.152655,10.277477,High
3,2024-03-01 00:15:00,40.725849,-73.980134,37,5.730536,37.695567,Red,Fog,0,-0.271965,66,10,410.384292,29.243279,High
4,2024-03-01 00:20:00,40.813265,-73.961631,64,61.348034,22.313358,Red,Snow,0,-0.797606,3,5,364.466342,16.801459,Low


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Timestamp               5000 non-null   datetime64[ns]
 1   Latitude                5000 non-null   float64       
 2   Longitude               5000 non-null   float64       
 3   Vehicle_Count           5000 non-null   int64         
 4   Traffic_Speed_kmh       5000 non-null   float64       
 5   Road_Occupancy_%        5000 non-null   float64       
 6   Traffic_Light_State     5000 non-null   object        
 7   Weather_Condition       5000 non-null   object        
 8   Accident_Report         5000 non-null   int64         
 9   Sentiment_Score         5000 non-null   float64       
 10  Ride_Sharing_Demand     5000 non-null   int64         
 11  Parking_Availability    5000 non-null   int64         
 12  Emission_Levels_g_km    5000 non-null   float64 

In [5]:
unique_locations = df[["Latitude", "Longitude"]].drop_duplicates()
print("Number of distinct GPS points:", len(unique_locations))

Number of distinct GPS points: 5000


In [6]:
print(df["Traffic_Condition"].value_counts(dropna=False))
print(df["Traffic_Condition"].value_counts(normalize=True))

Traffic_Condition
High      3166
Medium    1475
Low        359
Name: count, dtype: int64
Traffic_Condition
High      0.6332
Medium    0.2950
Low       0.0718
Name: proportion, dtype: float64


In [7]:
missing_summary = df.isnull().sum().sort_values(ascending=False)
print(missing_summary)

Timestamp                 0
Latitude                  0
Longitude                 0
Vehicle_Count             0
Traffic_Speed_kmh         0
Road_Occupancy_%          0
Traffic_Light_State       0
Weather_Condition         0
Accident_Report           0
Sentiment_Score           0
Ride_Sharing_Demand       0
Parking_Availability      0
Emission_Levels_g_km      0
Energy_Consumption_L_h    0
Traffic_Condition         0
dtype: int64


In [8]:
df["Hour_Of_Day"] = df["Timestamp"].dt.hour
df["DateOnly"]   = df["Timestamp"].dt.date

In [9]:
agg_funcs = {
    "Vehicle_Count": "sum",
    "Traffic_Speed_kmh": "mean",
    "Road_Occupancy_%": "mean",
    "Traffic_Light_State": lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0],
    "Weather_Condition":   lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0],
    "Accident_Report":     "max",   # 1 if any accident in the hour
    "Sentiment_Score":     "mean",
    "Ride_Sharing_Demand": "sum",
    "Parking_Availability": "mean",
    "Emission_Levels_g_km": "sum",
    "Energy_Consumption_L_h": "mean",
    "Traffic_Condition":     lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0]
}

hourly_df = df.groupby(
    ["DateOnly", "Hour_Of_Day", "Latitude", "Longitude"],
    as_index=False
).agg(agg_funcs)

In [10]:
print(hourly_df.shape)  # How many “location‐hour” units?
print(hourly_df.head())
print(hourly_df.describe(include="all"))

(5000, 16)
     DateOnly  Hour_Of_Day   Latitude  Longitude  Vehicle_Count  \
0  2024-03-01            0  40.641112 -73.825095            125   
1  2024-03-01            0  40.717006 -73.789562             67   
2  2024-03-01            0  40.722495 -73.830212            117   
3  2024-03-01            0  40.725849 -73.980134             37   
4  2024-03-01            0  40.793534 -73.983809             12   

   Traffic_Speed_kmh  Road_Occupancy_% Traffic_Light_State Weather_Condition  \
0           9.150746         72.799903               Green             Clear   
1          73.905337         80.607375               Green             Clear   
2          60.481377         56.001738               Green               Fog   
3           5.730536         37.695567                 Red               Fog   
4          38.739874         69.653163               Green               Fog   

   Accident_Report  Sentiment_Score  Ride_Sharing_Demand  \
0                0        -0.594636          

In [16]:
def latlng_to_grid(lat, lng, deg_per_km=0.018):
    cell_lat = int(lat / deg_per_km)
    cell_lng = int(lng / deg_per_km)
    return (cell_lat, cell_lng)

hourly_df["Grid_Cell"] = hourly_df.apply(
    lambda row: latlng_to_grid(row["Latitude"], row["Longitude"]),
    axis=1
)

In [17]:
agg_by_cell = hourly_df.groupby(
    ["DateOnly", "Hour_Of_Day", "Grid_Cell"],
    as_index=False
).agg({
    "Vehicle_Count":            "sum",
    "Traffic_Speed_kmh":        "mean",
    "Road_Occupancy_%":         "mean",
    "Traffic_Light_State":      lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0],
    "Weather_Condition":        lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0],
    "Accident_Report":          "max",
    "Sentiment_Score":          "mean",
    "Ride_Sharing_Demand":      "sum",
    "Parking_Availability":     "mean",
    "Emission_Levels_g_km":     "sum",
    "Energy_Consumption_L_h":   "mean",
    "Traffic_Condition":        lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0]
})

In [18]:
print(agg_by_cell.shape)   # Number of (Date,Hour,Grid_Cell) units
print(agg_by_cell.head())  # First few rows
print(agg_by_cell.describe(include="all"))

(4909, 15)
     DateOnly  Hour_Of_Day      Grid_Cell  Vehicle_Count  Traffic_Speed_kmh  \
0  2024-03-01            0  (2257, -4101)            125           9.150746   
1  2024-03-01            0  (2262, -4110)             37           5.730536   
2  2024-03-01            0  (2262, -4101)            117          60.481377   
3  2024-03-01            0  (2262, -4099)             67          73.905337   
4  2024-03-01            0  (2266, -4110)             12          38.739874   

   Road_Occupancy_% Traffic_Light_State Weather_Condition  Accident_Report  \
0         72.799903               Green             Clear                0   
1         37.695567                 Red               Fog                0   
2         56.001738               Green               Fog                0   
3         80.607375               Green             Clear                0   
4         69.653163               Green               Fog                0   

   Sentiment_Score  Ride_Sharing_Demand  Park

In [19]:
num_unique_cells = agg_by_cell["Grid_Cell"].nunique()
num_unique_dates = agg_by_cell["DateOnly"].nunique()
print("Unique grid cells:", num_unique_cells)
print("Unique dates covered:", num_unique_dates)

Unique grid cells: 321
Unique dates covered: 18


In [20]:
agg_2km = agg_by_cell

In [22]:
import pandas as pd
import numpy as np

# 1. Define the new function for 2 km grid aggregation
def latlng_to_grid_2km(lat, lng, deg_per_km=0.018):
    """
    Convert a (lat, lng) pair to a 2 km-resolution grid cell.
    deg_per_km ≈ 0.018° in latitude/longitude ~ 2 km (since 1° ≈ 111 km).
    """
    cell_lat = int(lat / deg_per_km)
    cell_lng = int(lng / deg_per_km)
    return (cell_lat, cell_lng)

# 2. Apply this function to the original hourly DataFrame (hourly_df)
#    Note: if you no longer have 'hourly_df' (i.e., the raw, non-aggregated data),
#    you can re-read the CSV and re-create 'hourly_df' similarly to before. 
#    For now, let’s assume you still have 'hourly_df' which contains columns:
#      - 'Timestamp' (datetime),
#      - 'Latitude', 'Longitude',
#      - and all other sensor/social‐network features.

# If hourly_df is not in memory, reload it:
# hourly_df = pd.read_csv("smart_mobility_dataset.csv", parse_dates=["Timestamp"])
# (then re-create DateOnly, Hour_Of_Day as before)

# 3. Create the 2 km‐resolution "Grid_Cell_2km" column
hourly_df["Grid_Cell_2km"] = hourly_df.apply(
    lambda row: latlng_to_grid_2km(row["Latitude"], row["Longitude"]),
    axis=1
)

# 4. (Optional) Inspect the first few rows to confirm
hourly_df[["Latitude", "Longitude", "Grid_Cell_2km"]].head(10)


,Latitude,Longitude,Grid_Cell_2km
0,40.641112,-73.825095,"(2257, -4101)"
1,40.717006,-73.789562,"(2262, -4099)"
2,40.722495,-73.830212,"(2262, -4101)"
3,40.725849,-73.980134,"(2262, -4110)"
4,40.793534,-73.983809,"(2266, -4110)"
5,40.797139,-73.797445,"(2266, -4099)"
6,40.812362,-73.801751,"(2267, -4100)"
7,40.813265,-73.961631,"(2267, -4108)"
8,40.819549,-73.732462,"(2267, -4096)"
9,40.831119,-73.987354,"(2268, -4110)"


In [24]:
agg_2km = hourly_df.groupby(
    ["DateOnly", "Hour_Of_Day", "Grid_Cell_2km"],
    as_index=False
).agg({
    "Vehicle_Count":            "sum",   # sum of all vehicles detected in that 2 km cell and hour
    "Traffic_Speed_kmh":        "mean",  # average speed
    "Road_Occupancy_%":         "mean",  # average road occupancy
    "Traffic_Light_State":      lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0],
    "Weather_Condition":        lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0],
    "Accident_Report":          "max",   # if any accident occurred anywhere in that zone & hour
    "Sentiment_Score":          "mean",  # average social‐media sentiment
    "Ride_Sharing_Demand":      "sum",   # total ride‐sharing requests
    "Parking_Availability":     "mean",  # average parking slots available
    "Emission_Levels_g_km":     "sum",   # total emissions (in g/km) summed over all vehicles
    "Energy_Consumption_L_h":   "mean",  # average fuel/diesel usage
    "Traffic_Condition":        lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0]
})


In [25]:
print("New (2 km) aggregated shape:", agg_2km.shape)

# How many distinct 2 km grid cells?
unique_cells_2km = agg_2km["Grid_Cell_2km"].nunique()
print("Unique 2 km grid cells:", unique_cells_2km)

# How many distinct DateOnly values remain?
unique_dates_2km = agg_2km["DateOnly"].nunique()
print("Unique dates (should stay 18):", unique_dates_2km)


New (2 km) aggregated shape: (4909, 15)
Unique 2 km grid cells: 321
Unique dates (should stay 18): 18


In [26]:
# Show a few rows of the aggregation keys
print(agg_2km.loc[:, ["DateOnly", "Hour_Of_Day", "Grid_Cell_2km"]].head(10))

# Confirm no duplicates
duplicate_count = agg_2km.duplicated(subset=["DateOnly", "Hour_Of_Day", "Grid_Cell_2km"]).sum()
print("Number of duplicate (DateOnly, Hour, Grid_Cell_2km) rows:", duplicate_count)


     DateOnly  Hour_Of_Day  Grid_Cell_2km
0  2024-03-01            0  (2257, -4101)
1  2024-03-01            0  (2262, -4110)
2  2024-03-01            0  (2262, -4101)
3  2024-03-01            0  (2262, -4099)
4  2024-03-01            0  (2266, -4110)
5  2024-03-01            0  (2266, -4099)
6  2024-03-01            0  (2267, -4108)
7  2024-03-01            0  (2267, -4100)
8  2024-03-01            0  (2267, -4096)
9  2024-03-01            0  (2268, -4110)
Number of duplicate (DateOnly, Hour, Grid_Cell_2km) rows: 0


In [27]:
# # Observations per DateOnly
counts_by_date_2km = agg_2km["DateOnly"].value_counts().sort_index()
print("Cell‐hours per date (2 km):\n", counts_by_date_2km, "\n")

# # Observations per Hour_Of_Day
counts_by_hour_2km = agg_2km["Hour_Of_Day"].value_counts().sort_index()
print("Cell‐hours per hour of day (2 km):\n", counts_by_hour_2km)


Cell‐hours per date (2 km):
 DateOnly
2024-03-01    281
2024-03-02    284
2024-03-03    284
2024-03-04    287
2024-03-05    283
2024-03-06    280
2024-03-07    283
2024-03-08    283
2024-03-09    285
2024-03-10    282
2024-03-11    279
2024-03-12    285
2024-03-13    279
2024-03-14    283
2024-03-15    282
2024-03-16    284
2024-03-17    285
2024-03-18    100
Name: count, dtype: int64 

Cell‐hours per hour of day (2 km):
 Hour_Of_Day
0     213
1     210
2     213
3     213
4     213
5     210
6     214
7     212
8     210
9     201
10    200
11    197
12    200
13    196
14    199
15    200
16    199
17    202
18    201
19    201
20    200
21    201
22    203
23    201
Name: count, dtype: int64


In [28]:
# Re‐check data types
print(agg_2km.info())

# Re‐check null counts
print("Null counts in 2 km dataset:\n", agg_2km.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4909 entries, 0 to 4908
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   DateOnly                4909 non-null   object 
 1   Hour_Of_Day             4909 non-null   int32  
 2   Grid_Cell_2km           4909 non-null   object 
 3   Vehicle_Count           4909 non-null   int64  
 4   Traffic_Speed_kmh       4909 non-null   float64
 5   Road_Occupancy_%        4909 non-null   float64
 6   Traffic_Light_State     4909 non-null   object 
 7   Weather_Condition       4909 non-null   object 
 8   Accident_Report         4909 non-null   int64  
 9   Sentiment_Score         4909 non-null   float64
 10  Ride_Sharing_Demand     4909 non-null   int64  
 11  Parking_Availability    4909 non-null   float64
 12  Emission_Levels_g_km    4909 non-null   float64
 13  Energy_Consumption_L_h  4909 non-null   float64
 14  Traffic_Condition       4909 non-null   

In [29]:
# Traffic_Light_State levels
print("Traffic_Light_State levels:\n", agg_2km["Traffic_Light_State"].value_counts(), "\n")

# Weather_Condition levels
print("Weather_Condition levels:\n", agg_2km["Weather_Condition"].value_counts(), "\n")

# Traffic_Condition levels & proportions
print("Traffic_Condition levels:\n", agg_2km["Traffic_Condition"].value_counts())
print("Traffic_Condition proportions:\n", agg_2km["Traffic_Condition"].value_counts(normalize=True))


Traffic_Light_State levels:
 Traffic_Light_State
Yellow    1673
Green     1644
Red       1592
Name: count, dtype: int64 

Weather_Condition levels:
 Weather_Condition
Rain     1246
Fog      1231
Snow     1218
Clear    1214
Name: count, dtype: int64 

Traffic_Condition levels:
 Traffic_Condition
High      3132
Medium    1424
Low        353
Name: count, dtype: int64
Traffic_Condition proportions:
 Traffic_Condition
High      0.638012
Medium    0.290079
Low       0.071909
Name: proportion, dtype: float64


In [30]:
cont_cols = [
    "Vehicle_Count", "Traffic_Speed_kmh", "Road_Occupancy_%", 
    "Sentiment_Score", "Ride_Sharing_Demand", "Parking_Availability", 
    "Emission_Levels_g_km", "Energy_Consumption_L_h"
]
outlier_summary_2km = {}
for col in cont_cols:
    Q1 = agg_2km[col].quantile(0.25)
    Q3 = agg_2km[col].quantile(0.75)
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
    mask = (agg_2km[col] < lower) | (agg_2km[col] > upper)
    outlier_summary_2km[col] = (mask.sum(), 100 * mask.sum() / agg_2km.shape[0])

for col, (cnt, pct) in outlier_summary_2km.items():
    print(f"{col} (2 km): {cnt} outliers ({pct:.2f}%)")

Vehicle_Count (2 km): 7 outliers (0.14%)
Traffic_Speed_kmh (2 km): 0 outliers (0.00%)
Road_Occupancy_% (2 km): 0 outliers (0.00%)
Sentiment_Score (2 km): 0 outliers (0.00%)
Ride_Sharing_Demand (2 km): 15 outliers (0.31%)
Parking_Availability (2 km): 0 outliers (0.00%)
Emission_Levels_g_km (2 km): 14 outliers (0.29%)
Energy_Consumption_L_h (2 km): 0 outliers (0.00%)


In [31]:
cell_counts_2km = agg_2km["Grid_Cell_2km"].value_counts()
print(cell_counts_2km.describe())            # five-number summary
print("Cells with < 3 total hours:", (cell_counts_2km < 3).sum())

count    321.000000
mean      15.292835
std        6.315772
min        1.000000
25%       12.000000
50%       16.000000
75%       20.000000
max       32.000000
Name: count, dtype: float64
Cells with < 3 total hours: 14


In [32]:
# 2.1  Identify which 2 km cells are too sparse (< 3 “cell‐hour” rows)
cell_counts_2km = agg_2km["Grid_Cell_2km"].value_counts()
sparse_cells = cell_counts_2km[cell_counts_2km < 3].index.tolist()

print("Number of sparse cells to drop:", len(sparse_cells))  # Expect 14
print("Sample of sparse cells:", sparse_cells[:5])

# 2.2  Filter them out from agg_2km
agg_final = agg_2km[~agg_2km["Grid_Cell_2km"].isin(sparse_cells)].copy()

# 2.3  Verify new shape and unique cell count
print("After dropping sparse cells, new shape:", agg_final.shape)
print("After dropping, unique 2 km cells:", agg_final["Grid_Cell_2km"].nunique())

Number of sparse cells to drop: 14
Sample of sparse cells: [(2266, -4111), (2271, -4111), (2264, -4111), (2262, -4111), (2269, -4111)]
After dropping sparse cells, new shape: (4886, 15)
After dropping, unique 2 km cells: 307


In [33]:
# 3.1  Confirm no remaining grid cell has < 3 observations
new_counts = agg_final["Grid_Cell_2km"].value_counts()
print(new_counts.describe())  # min should now be ≥ 3

print("Cells with < 3 total hours (should be 0):", (new_counts < 3).sum())

# 3.2  Confirm no nulls and data types are intact
print(agg_final.info())
print("Null counts:\n", agg_final.isnull().sum())

# 3.3  Re‐confirm categorical distributions still balanced
print("Traffic_Light_State levels:\n", agg_final["Traffic_Light_State"].value_counts(), "\n")
print("Weather_Condition levels:\n", agg_final["Weather_Condition"].value_counts(), "\n")
print("Traffic_Condition proportions:\n", agg_final["Traffic_Condition"].value_counts(normalize=True))

count    307.000000
mean      15.915309
std        5.726272
min        3.000000
25%       13.000000
50%       17.000000
75%       20.000000
max       32.000000
Name: count, dtype: float64
Cells with < 3 total hours (should be 0): 0
<class 'pandas.core.frame.DataFrame'>
Index: 4886 entries, 0 to 4908
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   DateOnly                4886 non-null   object 
 1   Hour_Of_Day             4886 non-null   int32  
 2   Grid_Cell_2km           4886 non-null   object 
 3   Vehicle_Count           4886 non-null   int64  
 4   Traffic_Speed_kmh       4886 non-null   float64
 5   Road_Occupancy_%        4886 non-null   float64
 6   Traffic_Light_State     4886 non-null   object 
 7   Weather_Condition       4886 non-null   object 
 8   Accident_Report         4886 non-null   int64  
 9   Sentiment_Score         4886 non-null   float64
 10  Ride_Sharing_Demand     4

In [34]:
# 4.1  Create a new DateTime_Hourly column
agg_final["DateTime_Hourly"] = pd.to_datetime(
    agg_final["DateOnly"].astype(str) + " " + agg_final["Hour_Of_Day"].astype(str) + ":00:00"
)

# 4.2  Sort by this column (chronological order)
agg_final = agg_final.sort_values("DateTime_Hourly").reset_index(drop=True)

# Confirm sorting
print("First few rows after sort:\n", agg_final[["DateTime_Hourly", "Grid_Cell_2km", "Vehicle_Count"]].head())
print("Last few rows after sort:\n", agg_final[["DateTime_Hourly", "Grid_Cell_2km", "Vehicle_Count"]].tail())

First few rows after sort:
   DateTime_Hourly  Grid_Cell_2km  Vehicle_Count
0      2024-03-01  (2257, -4101)            125
1      2024-03-01  (2270, -4096)             12
2      2024-03-01  (2269, -4094)            205
3      2024-03-01  (2268, -4110)            202
4      2024-03-01  (2267, -4100)            130
Last few rows after sort:
          DateTime_Hourly  Grid_Cell_2km  Vehicle_Count
4881 2024-03-18 08:00:00  (2261, -4107)            257
4882 2024-03-18 08:00:00  (2262, -4097)             20
4883 2024-03-18 08:00:00  (2264, -4102)            147
4884 2024-03-18 08:00:00  (2265, -4102)             88
4885 2024-03-18 08:00:00  (2268, -4109)            217


In [35]:
N_total   = len(agg_final)
train_end = int(0.70 * N_total)
valid_end = int(0.85 * N_total)

train_df = agg_final.iloc[:train_end].copy()
valid_df = agg_final.iloc[train_end:valid_end].copy()
test_df  = agg_final.iloc[valid_end:].copy()

print("Total rows:", N_total)
print("Train rows:", len(train_df))
print("Valid rows:", len(valid_df))
print("Test rows:",  len(test_df))

Total rows: 4886
Train rows: 3420
Valid rows: 733
Test rows: 733


In [36]:
for name, subset in [("Train", train_df), ("Valid", valid_df), ("Test", test_df)]:
    print(f"\n{name} class proportions (Traffic_Condition):")
    print(subset["Traffic_Condition"].value_counts(normalize=True))



Train class proportions (Traffic_Condition):
Traffic_Condition
High      0.636550
Medium    0.289181
Low       0.074269
Name: proportion, dtype: float64

Valid class proportions (Traffic_Condition):
Traffic_Condition
High      0.654843
Medium    0.282401
Low       0.062756
Name: proportion, dtype: float64

Test class proportions (Traffic_Condition):
Traffic_Condition
High      0.623465
Medium    0.306958
Low       0.069577
Name: proportion, dtype: float64


In [37]:
# =============================================================================
# (1) Recompute Lag-1 Features on Full DataFrame, then Chronological Split
# =============================================================================

import pandas as pd
import numpy as np

# 1.1) Load or reference your final aggregated DataFrame "agg_final"
# If saved to CSV, uncomment and adjust the following:
# agg_final = pd.read_csv("agg_final.csv", parse_dates=["DateTime_Hourly"])

# Ensure 'agg_final' is in memory. Then:
agg_final["DateTime_Hourly"] = pd.to_datetime(agg_final["DateTime_Hourly"])
agg_final = agg_final.sort_values(["Grid_Cell_2km", "DateTime_Hourly"]).reset_index(drop=True)

# 1.2) Create Lag-1 Features grouped by Grid_Cell_2km
grouped = agg_final.groupby("Grid_Cell_2km", group_keys=False)
agg_final["Vehicle_Count_Lag1"]        = grouped["Vehicle_Count"].shift(1)
agg_final["Occupancy_Lag1"]            = grouped["Road_Occupancy_%"].shift(1)
agg_final["Sentiment_Lag1"]            = grouped["Sentiment_Score"].shift(1)
agg_final["RideShare_Lag1"]            = grouped["Ride_Sharing_Demand"].shift(1)
agg_final["Parking_Lag1"]              = grouped["Parking_Availability"].shift(1)
agg_final["Emissions_Lag1"]            = grouped["Emission_Levels_g_km"].shift(1)

# Drop rows missing any of the lagged values (these correspond to the first record of each grid cell)
pre_drop_count = len(agg_final)
agg_lagged = agg_final.dropna(subset=[
    "Vehicle_Count_Lag1", "Occupancy_Lag1", 
    "Sentiment_Lag1", "RideShare_Lag1", 
    "Parking_Lag1", "Emissions_Lag1"
]).reset_index(drop=True)
dropped = pre_drop_count - len(agg_lagged)
print(f"Dropped {dropped} rows due to missing lagged values. Remaining rows: {len(agg_lagged)}")

# 1.3) Sort chronologically by DateTime_Hourly before splitting
agg_lagged = agg_lagged.sort_values("DateTime_Hourly").reset_index(drop=True)

# 1.4) Chronological 70%-15%-15% Train/Validation/Test Split
N = len(agg_lagged)
train_end = int(0.70 * N)
valid_end = int(0.85 * N)

train_full = agg_lagged.iloc[:train_end].copy().reset_index(drop=True)
valid_full = agg_lagged.iloc[train_end:valid_end].copy().reset_index(drop=True)
test_full  = agg_lagged.iloc[valid_end:].copy().reset_index(drop=True)

print(f"Split rows → Train: {len(train_full)}, Valid: {len(valid_full)}, Test: {len(test_full)}")

Dropped 307 rows due to missing lagged values. Remaining rows: 4579
Split rows → Train: 3205, Valid: 687, Test: 687


In [38]:
# =============================================================================
# (2) Feature Engineering for Hypothesis Testing
# =============================================================================

# 2.1) Create DayOfWeek from DateTime_Hourly
for df_ in [train_full, valid_full, test_full]:
    df_["DayOfWeek"] = df_["DateTime_Hourly"].dt.day_name()

# 2.2) Create binary flags aligned with our hypotheses:
#     - Sentiment_Negative: 1 if Sentiment_Score < -0.2
#     - Emission_High: 1 if Emission_Levels_g_km >= 150
#     - Weather_Adverse: 1 if Weather_Condition is 'Rain' or 'Snow'
# Accident_Report is already 0/1 for an accident in current hour.
for df_ in [train_full, valid_full, test_full]:
    df_["Sentiment_Negative"] = (df_["Sentiment_Score"] < -0.2).astype(int)
    df_["Emission_High"]      = (df_["Emission_Levels_g_km"] >= 150).astype(int)
    df_["Weather_Adverse"]    = df_["Weather_Condition"].isin(["Rain", "Snow"]).astype(int)

# 2.3) One-hot encode categorical variables (dropping first level to avoid multicollinearity):
#     - Traffic_Light_State (Red/Yellow/Green → create 2 dummy columns)
#     - Weather_Condition (Clear/Fog/Rain/Snow → create 3 dummy columns)
#     - Hour_Of_Day (0–23 → create 23 dummy columns)
#     - DayOfWeek (Monday–Sunday → create 6 dummy columns)
# We will NOT create dummies for Grid_Cell_2km in the regression to avoid extremely high dimensionality.
def create_feature_matrix(df):
    # Numeric columns to include
    numeric_cols = [
        "Vehicle_Count_Lag1", "Occupancy_Lag1", "Sentiment_Lag1", 
        "RideShare_Lag1", "Parking_Lag1", "Emissions_Lag1", 
        "Accident_Report", "Emission_High", "Weather_Adverse"
    ]
    num_feats = df[numeric_cols].copy()
    
    # One-hot encoding for categorical variables
    cat_feats = pd.get_dummies(df[[
        "Traffic_Light_State", "Weather_Condition", "Hour_Of_Day", "DayOfWeek"
    ]].astype(str), drop_first=True)
    
    # Concatenate numeric and categorical features
    X = pd.concat([num_feats.reset_index(drop=True), cat_feats.reset_index(drop=True)], axis=1)
    return X

X_train = create_feature_matrix(train_full)
X_valid = create_feature_matrix(valid_full)
X_test  = create_feature_matrix(test_full)

# 2.4) Prepare target variable: Traffic_Condition → numeric codes (0=Low, 1=Medium, 2=High)
mapping = {"Low": 0, "Medium": 1, "High": 2}
y_train = train_full["Traffic_Condition"].map(mapping).astype(int)
y_valid = valid_full["Traffic_Condition"].map(mapping).astype(int)
y_test  = test_full["Traffic_Condition"].map(mapping).astype(int)

In [43]:
# ----------------------------------------------------------------------------
# (A) Check dtypes in X_train, X_valid, X_test
# ----------------------------------------------------------------------------
print("---- X_train dtypes ----")
print(X_train.dtypes.value_counts(), "\n")
print("Columns still non-numeric (if any):")
print([col for col, dt in X_train.dtypes.items() if dt == "object"], "\n")

print("---- X_valid dtypes ----")
print(X_valid.dtypes.value_counts(), "\n")
print("Columns still non-numeric (if any):")
print([col for col, dt in X_valid.dtypes.items() if dt == "object"], "\n")

print("---- X_test dtypes ----")
print(X_test.dtypes.value_counts(), "\n")
print("Columns still non-numeric (if any):")
print([col for col, dt in X_test.dtypes.items() if dt == "object"])


---- X_train dtypes ----
bool       34
float64     6
int64       3
Name: count, dtype: int64 

Columns still non-numeric (if any):
[] 

---- X_valid dtypes ----
bool       30
float64     6
int64       3
Name: count, dtype: int64 

Columns still non-numeric (if any):
[] 

---- X_test dtypes ----
bool       31
float64     6
int64       3
Name: count, dtype: int64 

Columns still non-numeric (if any):
[]


In [44]:
import statsmodels.api as sm

# (1) Add a constant to X_train, X_valid, X_test
X_train_sm = sm.add_constant(X_train, has_constant="add")
X_valid_sm = sm.add_constant(X_valid, has_constant="add")
X_test_sm  = sm.add_constant(X_test,  has_constant="add")

# (2) Verify that all dtypes are numeric after adding “const”
unique_dtypes = set(X_train_sm.dtypes.values)
print("Unique dtypes in X_train_sm:", unique_dtypes)
# You should see only {'float64'} or maybe {'float64','int64'} – but no 'object'.

Unique dtypes in X_train_sm: {dtype('bool'), dtype('float64'), dtype('int64')}


In [46]:
mapping = {"Low": 0, "Medium": 1, "High": 2}
y_train = train_full["Traffic_Condition"].map(mapping).astype(int)
y_valid = valid_full["Traffic_Condition"].map(mapping).astype(int)
y_test  = test_full["Traffic_Condition"].map(mapping).astype(int)

print("Train label distribution:")
print(y_train.value_counts(normalize=True).round(3))
print("\nValid label distribution:")
print(y_valid.value_counts(normalize=True).round(3))
print("\nTest label distribution:")
print(y_test.value_counts(normalize=True).round(3))

Train label distribution:
Traffic_Condition
2    0.641
1    0.288
0    0.072
Name: proportion, dtype: float64

Valid label distribution:
Traffic_Condition
2    0.648
1    0.285
0    0.067
Name: proportion, dtype: float64

Test label distribution:
Traffic_Condition
2    0.623
1    0.307
0    0.070
Name: proportion, dtype: float64


In [48]:
import statsmodels.api as sm
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 1.1) Add an intercept (“const”)—Statsmodels does not do this automatically
X_train_sm = sm.add_constant(X_train, has_constant="add")
X_valid_sm = sm.add_constant(X_valid, has_constant="add")
X_test_sm  = sm.add_constant(X_test,  has_constant="add")

# 1.2) Cast every column to float64 (so that no column is bool or int or object)
X_train_sm = X_train_sm.astype("float64")
X_valid_sm = X_valid_sm.astype("float64")
X_test_sm  = X_test_sm.astype("float64")

# 1.3) Confirm all dtypes are now float64
print("Unique dtypes in X_train_sm:", set(X_train_sm.dtypes.values))
print("Unique dtypes in X_valid_sm:", set(X_valid_sm.dtypes.values))
print("Unique dtypes in X_test_sm: ", set(X_test_sm.dtypes.values))

# 1.4) Ensure y’s are plain int
y_train = y_train.astype(int)
y_valid = y_valid.astype(int)
y_test  = y_test.astype(int)

print("\ny_train dtypes:", y_train.dtype, "\ny_valid dtype:", y_valid.dtype, "\ny_test dtype:", y_test.dtype)

Unique dtypes in X_train_sm: {dtype('float64')}
Unique dtypes in X_valid_sm: {dtype('float64')}
Unique dtypes in X_test_sm:  {dtype('float64')}

y_train dtypes: int64 
y_valid dtype: int64 
y_test dtype: int64


In [52]:
# ----------------------------------------------------------------------------
# (3) Fit statsmodels’ MNLogit on the training data
# ----------------------------------------------------------------------------

# 3.1) Instantiate the MNLogit model
mnlogit_model = sm.MNLogit(endog=y_train, exog=X_train_sm)

# 3.2) Fit with Newton-Raphson (maxiter=100, silent output)
mnlogit_result = mnlogit_model.fit(
    method="newton", 
    maxiter=100, 
    full_output=True, 
    disp=False
)

# 3.3) Print a summary table
print(mnlogit_result.summary())


                          MNLogit Regression Results                          
Dep. Variable:      Traffic_Condition   No. Observations:                 3205
Model:                        MNLogit   Df Residuals:                     3119
Method:                           MLE   Df Model:                           84
Date:                Sat, 31 May 2025   Pseudo R-squ.:                 0.07011
Time:                        14:40:38   Log-Likelihood:                -2481.9
converged:                      False   LL-Null:                       -2669.1
Covariance Type:            nonrobust   LLR p-value:                 2.936e-38
       Traffic_Condition=1       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                          1.2472      0.563      2.217      0.027       0.145       2.350
Vehicle_Count_Lag1             0.0016      0.001      1.771      0.077      -0.000 

/Users/ethanyan01/Desktop/c-analysis/.venv/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/ethanyan01/Desktop/c-analysis/.venv/lib/python3.13/site-packages/statsmodels/discrete/discrete_model.py:5475: RuntimeWarning: invalid value encountered in sqrt
  bse = np.sqrt(np.diag(self.cov_params()))


In [51]:
# ───────────────────────────────────────────────────────────────────────────
# (A) After you add the constant to X_train, X_valid, X_test, and cast to float:
# ───────────────────────────────────────────────────────────────────────────

# 1) Add constant + cast to float64 (as you already did)
X_train_sm = sm.add_constant(X_train, has_constant="add").astype("float64")
X_valid_sm = sm.add_constant(X_valid, has_constant="add").astype("float64")
X_test_sm  = sm.add_constant(X_test,  has_constant="add").astype("float64")

# 2) Reindex validation & test so they share exactly the same columns (and order) as X_train_sm
X_valid_sm = X_valid_sm.reindex(columns=X_train_sm.columns, fill_value=0.0)
X_test_sm  = X_test_sm.reindex(columns=X_train_sm.columns, fill_value=0.0)

# 3) Double‐check that every column now lines up (train vs. valid vs. test)
print("X_train_sm shape:", X_train_sm.shape)
print("X_valid_sm shape:", X_valid_sm.shape)
print("X_test_sm  shape:", X_test_sm.shape)
print("Column mismatch (train vs. valid)?", list(X_train_sm.columns.symmetric_difference(X_valid_sm.columns)))
print("Column mismatch (train vs. test)?",  list(X_train_sm.columns.symmetric_difference(X_test_sm.columns)))

X_train_sm shape: (3205, 44)
X_valid_sm shape: (687, 44)
X_test_sm  shape: (687, 44)
Column mismatch (train vs. valid)? []
Column mismatch (train vs. test)? []


In [53]:
# 3.1) Compute predicted probabilities on validation set
probs_valid = mnlogit_result.predict(X_valid_sm)
# `probs_valid` is an (n_valid × 3) array: columns [P(Low), P(Medium), P(High)]

# 3.2) Convert to predicted label by argmax
y_valid_pred = np.argmax(probs_valid, axis=1)

# 3.3) Compute accuracy, confusion matrix, and classification report
acc_valid = accuracy_score(y_valid, y_valid_pred)
print(f"Validation accuracy (MNLogit): {acc_valid:.3f}\n")

print("Confusion matrix (true rows vs. predicted columns):")
print(confusion_matrix(y_valid, y_valid_pred))

print("\nDetailed classification report:")
print(classification_report(y_valid, y_valid_pred, target_names=["Low","Medium","High"]))


Validation accuracy (MNLogit): 0.646

Confusion matrix (true rows vs. predicted columns):
[[  0   0  46]
 [  0   2 194]
 [  0   3 442]]

Detailed classification report:
              precision    recall  f1-score   support

         Low       0.00      0.00      0.00        46
      Medium       0.40      0.01      0.02       196
        High       0.65      0.99      0.78       445

    accuracy                           0.65       687
   macro avg       0.35      0.33      0.27       687
weighted avg       0.53      0.65      0.51       687



/Users/ethanyan01/Desktop/c-analysis/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ethanyan01/Desktop/c-analysis/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ethanyan01/Desktop/c-analysis/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [54]:
# =============================================================================
# (IV) Final check on the test set
# =============================================================================

probs_test   = mnlogit_result.predict(X_test_sm)
y_test_pred  = np.argmax(probs_test, axis=1)

acc_test = accuracy_score(y_test, y_test_pred)
print(f"\nTest accuracy (MNLogit): {acc_test:.3f}\n")

print("Test confusion matrix:")
print(confusion_matrix(y_test, y_test_pred))

print("\nTest classification report:")
print(classification_report(y_test, y_test_pred, target_names=["Low","Medium","High"]))


Test accuracy (MNLogit): 0.620

Test confusion matrix:
[[  0   0  48]
 [  0   0 211]
 [  0   2 426]]

Test classification report:
              precision    recall  f1-score   support

         Low       0.00      0.00      0.00        48
      Medium       0.00      0.00      0.00       211
        High       0.62      1.00      0.77       428

    accuracy                           0.62       687
   macro avg       0.21      0.33      0.26       687
weighted avg       0.39      0.62      0.48       687



/Users/ethanyan01/Desktop/c-analysis/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ethanyan01/Desktop/c-analysis/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ethanyan01/Desktop/c-analysis/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif